<a href="https://colab.research.google.com/github/zcongfly/huggingface-nlp-learning-note/blob/main/10_%E4%B8%AD%E6%96%87NLP%E4%BB%BB%E5%8A%A1%E5%AE%9E%E6%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] datasets

## 中文分类任务

In [ ]:
import torch
from datasets import load_dataset

#定义数据集
class Dataset(torch.utils.data.Dataset):

    def __init__(self, split):
        self.dataset = load_dataset(path='seamew/ChnSentiCorp', split=split)
        # self.dataset = load_from_disk('seamew/ChnSentiCorp',split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']

        return text, label

raw_dataset=Dataset('train')
len(raw_dataset), raw_dataset[0]

(9600,
 ('选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  1))

In [ ]:
from transformers import BertTokenizer

tokenizer=BertTokenizer.from_pretrained("bert-base-chinese")

# 定义批处理函数
def collate_fn(dataset):
    sentences=[data[0] for data in dataset]
    labels=[data[1] for data in dataset]

    # 编码
    tokenized_dataset=tokenizer.batch_encode_plus(
        batch_text_or_text_pairs=sentences,
        truncation=True,
        padding='max_length',
        max_length=500,
        return_tensors='pt',
        return_length=True
    )

    input_ids = tokenized_dataset['input_ids']
    attention_mask = tokenized_dataset['attention_mask']
    token_type_ids = tokenized_dataset['token_type_ids']
    labels = torch.LongTensor(labels)

    return input_ids, attention_mask, token_type_ids, labels

`BertTokenizer(texts, padding=True, truncation=True, return_tensors='pt')`和`BertTokenizer.batch_encode_plus`都是用于对多个文本进行批量编码的方式，但它们也有一些区别。

- 返回值不同：`BertTokenizer(texts, padding=True, truncation=True, return_tensors='pt')`直接返回编码结果的PyTorch张量，而`BertTokenizer.batch_encode_plus`返回一个字典，其中包含了所有文本的编码结果。
- 参数不完全相同：两种方法都支持`padding`和`truncation`参数，但`BertTokenizer()`额外支持了`max_length`、`add_special_tokens`等参数；而`BertTokenizer.batch_encode_plus`则额外支持了`return_attention_mask`和`return_token_type_ids`等参数。

另外，虽然这两种方法不能同时使用，但它们实现的功能是相同的。因为在`BertTokenizer()`中，如果为多个文本提供输入，则它会自动将它们转换为`BertTokenizer.batch_encode_plus`可以接受的格式，并将结果返回为PyTorch张量。

需要注意，与`BertTokenizer.batch_encode_plus`相比，`BertTokenizer()`不能处理超过最大长度限制的文本，而是通过截断文本或者对文本进行填充的方式来处理这些文本。同时，`BertTokenizer()`也可能会自动添加特殊标记（如`[CLS]`、`[SEP]`等）。

因此，需要根据实际情况选择使用哪种方式。如果想要更细粒度地控制编码过程并获得更多的信息，可以使用`BertTokenizer.batch_encode_plus`；如果只是简单地对一组文本进行编码，可以使用`BertTokenizer()`，它的接口更加简单直接。

In [ ]:
# 定义数据加载器并查看数据样例
loader=torch.utils.data.DataLoader(
    dataset=raw_dataset,
    batch_size=16,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True
)

for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

600


(torch.Size([16, 500]),
 torch.Size([16, 500]),
 torch.Size([16, 500]),
 tensor([0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1]))

In [ ]:
from transformers import BertModel

# 加载预训练模型
pretrained_model = BertModel.from_pretrained('bert-base-chinese')

# 把预训练模型的梯度冻结住，不训练，不需要计算梯度
for param in pretrained_model.parameters():
    param.requires_grad_(False)

# 模型试算
out = pretrained_model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    token_type_ids=token_type_ids
)

out.last_hidden_state.shape

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 500, 768])

In [ ]:
# 定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc=torch.nn.Linear(768,2)

    def forward(self,input_ids,attention_mask,token_type_ids):
        with torch.no_grad():
            out = pretrained_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

        out=self.fc(out.last_hidden_state[:,0])
        out=out.softmax(dim=1)
        return out

model = Model()

```out=self.fc(out.last_hidden_state[:,0])```

bert模型为什么最后输出时候只需要第0个数据即可？

在Bert模型中，`last_hidden_state`这个张量的形状为`(batch_size, sequence_length, hidden_size)`。其中，`batch_size`表示当前批次输入样本的数量，`sequence_length`表示每个样本的序列长度，`hidden_size`表示隐藏层的维度大小。

`out.last_hidden_state[:, 0]`表示取`last_hidden_state`张量中所有输入序列中的第一个位置对应的向量，因为在Bert模型中，第一个位置通常是特殊标记符，如[CLS]或者[SEP]等，这些标记符表示了整个序列的语义信息。经过Bert的多层Transformer编码之后，这些位置对应的向量可以代表整个输入序列的含义。所以最后一步使用全连接层进行分类时，只需要使用这些向量进行分类即可，这就是为什么通常只取第一个位置的原因。

需要注意的是，在某些任务中，可能会使用除了第一个位置的其它位置的向量进行分类或者其它操作，这涉及到具体的任务和实现需求，需要根据具体情况来确定。

In [ ]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    out = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids
    )

    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 100:
        break

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0.727667510509491 0.4375
5 0.6662500500679016 0.5625
10 0.654441773891449 0.6875
15 0.5895128846168518 0.8125
20 0.5993697047233582 0.875
25 0.5442937016487122 1.0
30 0.5852233171463013 0.8125
35 0.5988757014274597 0.6875
40 0.5492776036262512 0.75
45 0.5926205515861511 0.8125
50 0.508979082107544 0.875
55 0.5518791675567627 0.75
60 0.5219196081161499 0.8125
65 0.6405618190765381 0.6875
70 0.4745098352432251 0.875
75 0.518297016620636 0.8125
80 0.5056362748146057 0.8125
85 0.45229312777519226 0.9375
90 0.5124148726463318 0.875
95 0.5208293795585632 0.8125
100 0.49476727843284607 0.9375


In [ ]:
# 测试
def test():
    model.eval()
    correct=0
    total=0

    loader_test=torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                               batch_size=32,
                                               collate_fn=collate_fn,
                                               shuffle=True,
                                               drop_last=True)
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):
        if i == 5:
            print(i)
            break

        with torch.no_grad():
            out=model(input_ids=input_ids,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids)
        out=out.argmax(dim=1)
        correct+=(out==labels).sum().item()
        total+=len(labels)
    print(correct / total)

test()

5
0.8


## 中文填空任务

In [ ]:
import torch
from datasets import load_dataset

# 定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self,split):
        dataset=load_dataset(path='seamew/ChnSentiCorp',split=split)

        def f(data):
            return len(data['text'])>30

        self.dataset=dataset.filter(f)  #过滤掉字符长度小于30的句子

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,i):
        text=self.dataset[i]['text']
        return text

dataset=Dataset('train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset chn_senti_corp downloaded and prepared to /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85. Subsequent calls will reuse this data.


Filter:   0%|          | 0/9600 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertTokenizer

# 加载字典和分词工具
tokenizer=BertTokenizer.from_pretrained('bert-base-chinese')
tokenizer

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [ ]:
def collate_fn(data):
    data=tokenizer.batch_encode_plus(batch_text_or_text_pairs=data,
                                     truncation=True,
                                     padding='max_length',
                                     max_length=30,
                                     return_tensors='pt',
                                     return_length=True)
    input_ids=data['input_ids']
    attention_mask=data['attention_mask']
    token_type_ids=data['token_type_ids']

    # 把第15个词替换成mask
    labels=input_ids[:,15].reshape(-1).clone()
    input_ids[:,15]=tokenizer.get_vocab()[tokenizer.mask_token]
    # print (data['length'], data['length']. max())

    return input_ids,attention_mask,token_type_ids,labels

In [ ]:
# 数据加载器
loader=torch.utils.data.DataLoader(dataset=dataset,
                                   batch_size=16,
                                   collate_fn=collate_fn,
                                   shuffle=True,
                                   drop_last=True)

for i, (input_ids,attention_mask,token_type_ids, labels) in enumerate(loader):
    break

print(len(loader))
print(tokenizer.decode(input_ids [0]))
print(tokenizer.decode(labels[0]))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape

574
[CLS] 服 务 员 素 质 很 高, 比 国 内 很 多 5 [MASK] 饭 店 要 好! 设 施 有 些 旧, 但 维 [SEP]
星


(torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16]))

In [ ]:
# 加载Bert中文模型
from transformers import BertModel

# 加载预训练模型
pretrained_model=BertModel.from_pretrained('bert-base-chinese')

# 冻结参数,不参与参数更新
for param in pretrained_model.parameters():
    param.requires_grad_(False)

# 模型试算
out=pretrained_model(input_ids=input_ids,
               attention_mask=attention_mask,
               token_type_ids=token_type_ids)

out.last_hidden_state.shape

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 30, 768])

In [ ]:
# 定义下游任务模型

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder=torch.nn.Linear(768,tokenizer.vocab_size,bias=False) #不使用默认偏置项
        self.bias=torch.nn.Parameter(torch.zeros(tokenizer.vocab_size)) #手动添加自定义偏置项
        self.decoder.bias=self.bias

    def forward(self,input_ids,attention_mask,token_type_ids):
        with torch.no_grad():
            out = pretrained_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

        out=self.decoder(out.last_hidden_state[:,15])
        return out

model=Model()
model(input_ids=input_ids,
       attention_mask=attention_mask,
       token_type_ids=token_type_ids).shape

torch.Size([16, 21128])

In [ ]:
model (input_ids=input_ids,
       attention_mask=attention_mask,
       token_type_ids=token_type_ids)

tensor([[ 0.4699, -0.3906, -0.3997,  ..., -0.0137,  0.5438, -1.0292],
        [ 0.3297, -1.2258, -0.7249,  ..., -0.1350,  0.2315, -0.1962],
        [ 0.3920,  0.0125, -0.6553,  ..., -0.0303,  0.6037,  0.1329],
        ...,
        [ 0.2990, -0.4178, -0.8610,  ..., -0.0759,  0.4767,  0.0653],
        [ 0.5332, -0.1695, -0.7683,  ..., -0.2023,  0.5060,  0.1468],
        [ 0.1174, -0.5266, -0.2589,  ..., -0.4839,  0.8058, -0.0337]],
       grad_fn=<AddmmBackward0>)

In [ ]:
from transformers import AdamW

# 训练
optimizer=AdamW(model.parameters(),lr=5e-4)
criterion=torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(5):
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
        out = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 50 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            print(epoch, i, loss.item(), accuracy)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0 10.238602638244629 0.0
0 50 8.496200561523438 0.125
0 100 5.623972415924072 0.375
0 150 6.083519458770752 0.0625
0 200 5.3640594482421875 0.3125
0 250 5.480146408081055 0.3125
0 300 4.695908546447754 0.4375
0 350 5.131112098693848 0.375
0 400 3.017376184463501 0.5
0 450 4.169382572174072 0.5
0 500 5.4901628494262695 0.1875
0 550 2.4733784198760986 0.6875
1 0 2.967989921569824 0.5
1 50 2.291616678237915 0.5625
1 100 1.8413182497024536 0.75
1 150 3.598939895629883 0.375
1 200 2.672093629837036 0.625
1 250 2.8248465061187744 0.625
1 300 1.7264764308929443 0.75
1 350 1.2783411741256714 0.8125
1 400 1.9367700815200806 0.625
1 450 1.5558221340179443 0.875
1 500 2.0046002864837646 0.5625
1 550 1.9868617057800293 0.75
2 0 1.07943594455719 0.8125
2 50 1.2917327880859375 0.75
2 100 0.8108772039413452 0.875
2 150 1.5752943754196167 0.625
2 200 1.0611276626586914 0.875
2 250 0.647756040096283 0.8125
2 300 2.1700284481048584 0.5625
2 350 0.748180627822876 0.9375
2 400 1.1434545516967773 0.6875


In [ ]:
# 测试
def test():
    model.eval()
    correct=0
    total=0

    loader_test=torch.utils.data.DataLoader(dataset=Dataset('test'),
                                               batch_size=32,
                                               collate_fn=collate_fn,
                                               shuffle=True,
                                               drop_last=True)
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):
        if i == 15:
            break

        print(i)

        with torch.no_grad():
            out=model(input_ids=input_ids,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids)
        out=out.argmax(dim=1)
        correct+=(out==labels).sum().item()
        total+=len(labels)

        print(tokenizer.decode(input_ids[0]))
        print(tokenizer.decode(out[0]),tokenizer.decode(labels[0]))
    print(correct / total)

test()

## 中文句子关系推断

In [9]:
import torch
from datasets import load_dataset
import random

# 定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self,split):
        dataset=load_dataset(path='seamew/ChnSentiCorp',split=split)

        def f(data):
            return len(data['text'])>40

        self.dataset=dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,i):
        text=self.dataset[i]['text']

        # 切分一句话为前半句和后半句
        sentence1=text[:20]
        sentence2=text[20:40]
        label=0

        # 有一半的概率把后半句替换成一句无关的话
        if random.randint(0,1)==0:
            j=random.randint(0,len(self.dataset)-1)
            sentence2=self.dataset[j]['text'][20:40]
            label=1

        return sentence1,sentence2,label

In [10]:
# 查看数据样例
dataset=Dataset('train')

sentence1,sentence2,label=dataset[0]
len(dataset),sentence1,sentence2,label

(8001, '选择珠江花园的原因就是方便，有电动扶梯直', '早餐好像和去年不一样了。我去的时候两个老', 1)

In [11]:
from transformers import BertTokenizer

# 加载字典和分词工具
tokenizer=BertTokenizer.from_pretrained('bert-base-chinese')
tokenizer

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [12]:
# 定义批处理函数

def collate_fn(data):
    sentences=[i[:2] for i in data]
    labels=[i[2] for i in data]

    # 编码
    data=tokenizer.batch_encode_plus(batch_text_or_text_pairs=sentences,
                                     truncation=True,
                                     padding='max_length',
                                     max_length=45,
                                     return_tensors='pt',
                                     return_length=True,
                                     add_special_tokens=True)
    #input_ids：编码之后的数字
    #attention_mask：是补零的位置是0，其他位置是1
    #token_type_ids：第一个句子和特殊符号的位置是0，第二个句子的位置是1
    input_ids=data['input_ids']
    attention_mask=data['attention_mask']
    token_type_ids=data['token_type_ids']
    labels=torch.LongTensor(labels)

    # print(data['length'],data['length'].max())
    return input_ids,attention_mask,token_type_ids,labels

In [13]:
# 定义数据加载器并查看数据样例

# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids,attention_mask,token_type_ids, labels) in enumerate(loader):
    break

print(len(loader))
print(tokenizer.decode(input_ids[0]))
input_ids.shape, attention_mask,token_type_ids.shape, labels.shape, labels

1000
[CLS] 前 几 次 和 朋 友 一 起 住 过 这 家 酒 店, 总 体 感 觉 还 [SEP] 挺 大 ， 而 且 好 像 不 能 调 ？ 似 乎 只 能 调 屏 幕 整 亮 [SEP] [PAD] [PAD]


(torch.Size([8, 45]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [25]:
from transformers import BertModel

pretrained_model=BertModel.from_pretrained('bert-base-chinese')

# 定义下游任务模型
class Model (torch. nn. Module):
    def __init__(self):
        super().__init__()
        self. fc = torch. nn. Linear (768, 2)

    def forward(self,input_ids,attention_mask,token_type_ids):
        with torch.no_grad():
            out = pretrained_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )
        out = self. fc (out. last_hidden_state[:, 0])
        out = out. softmax (dim=1)
        return out

model = Model ()
model (input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids). shape

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([8, 2])

In [26]:
from transformers import AdamW

# 训练
optimizer = AdamW(model. parameters(), lr=5e-4)
criterion = torch. nn. CrossEntropyLoss ()

model. train()
for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    out = model (input_ids=input_ids,
                 attention_mask=attention_mask,
                 token_type_ids=token_type_ids)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer. zero_grad()
    if i % 5 == 0:
        out = out.argmax (dim=1)
        accuracy = (out == labels).sum(). item() /len(labels)
        print(i, loss. item(), accuracy)
    if  i == 300:
        break

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0.6681267023086548 0.75
5 0.5991316437721252 0.875
10 0.591526210308075 0.75
15 0.6078758239746094 0.625
20 0.4053194522857666 1.0
25 0.4421483278274536 0.875
30 0.36066725850105286 1.0
35 0.5034912824630737 0.875
40 0.5311712622642517 0.75
45 0.702488899230957 0.625
50 0.49385830760002136 0.875
55 0.43100717663764954 0.875
60 0.38723668456077576 0.875
65 0.4061906039714813 0.875
70 0.39737728238105774 1.0
75 0.3624025881290436 1.0
80 0.37542444467544556 0.875
85 0.46156710386276245 0.875
90 0.4425033926963806 0.875
95 0.4976716935634613 0.875
100 0.5571668148040771 0.75
105 0.3884112536907196 1.0
110 0.571444571018219 0.75
115 0.4542813301086426 0.875
120 0.5250269770622253 0.75
125 0.375055730342865 1.0
130 0.5344651937484741 0.75
135 0.434628963470459 0.875
140 0.549827516078949 0.75
145 0.3366841673851013 1.0
150 0.362076997756958 1.0
155 0.3326747417449951 1.0
160 0.6500239372253418 0.5
165 0.39077502489089966 1.0
170 0.36040931940078735 1.0
175 0.5107102990150452 0.75
180 0.469

In [29]:
# 测试
def test():
    model.eval()
    correct=0
    total=0

    loader_test=torch.utils.data.DataLoader(dataset=Dataset('test'),
                                               batch_size=32,
                                               collate_fn=collate_fn,
                                               shuffle=True,
                                               drop_last=True)
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        if i == 5:
            break

        print(i)

        with torch.no_grad():
            out=model(input_ids=input_ids,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids)
        out=out.argmax(dim=1)
        correct+=(out==labels).sum().item()
        total+=len(labels)

        print(tokenizer.decode(input_ids[0]))
        print(tokenizer.decode(out[0]),tokenizer.decode(labels[0]))
    print(correct / total)

test()

Filter:   0%|          | 0/1200 [00:00<?, ? examples/s]

0
[CLS] 惠 普 这 款 整 机 配 置 均 衡 ， 内 存 稍 小 ， 京 东 送 个 [SEP] 内 存 就 太 好 了 。 本 本 外 观 简 约 ， 不 愧 为 商 用 型 [SEP] [PAD] [PAD]
[ P A D ] [ P A D ]
1
[CLS] 快 4 岁 的 女 儿 很 喜 欢! 同 时 买 的 还 有 《 女 孩 子 [SEP] 亲 人 一 个 个 死 去 了 ， 唯 有 他 还 活 着 ！ 真 是 太 累 [SEP] [PAD] [PAD]
[ u n u s e d 1 ] [ u n u s e d 1 ]
2
[CLS] 键 盘 设 计 不 太 合 理 ， ctrl 键 太 远 shi [SEP] 教 室 朝 南 ， 没 有 风 筝 》 的 麻 宁 ， 不 知 道 是 她 成 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[ u n u s e d 1 ] [ u n u s e d 1 ]
3
[CLS] 地 理 位 置 不 错 ， 房 间 环 境 也 可 以 ， 洗 衣 速 度 快 [SEP] ， 值 得 称 赞 ， 唯 一 不 足 就 是 装 修 期 间 ， 很 多 楼 [SEP] [PAD] [PAD]
[ P A D ] [ P A D ]
4
[CLS] 简 单 ， 大 方 ， 在 同 类 尺 寸 的 款 型 的 笔 记 本 中 不 [SEP] 显 厚 重 ， 轻 薄 感 ！ 很 安 静 ， 几 乎 没 有 声 音 ， 音 [SEP] [PAD] [PAD]
[ P A D ] [ P A D ]
0.84375
